In [1]:
#Imports
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import csv

In [2]:
#Get function to get the content html content from the provided url
def simple_get(url):
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

#Function that checks if the response received was good or not
def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

#Function that prints errors
def log_error(e):
    print(e)

In [3]:
#Get the HTML response
raw_html = simple_get('https://money.cnn.com/data/hotstocks/')

In [4]:
from bs4 import BeautifulSoup  #import beautiful soup library to process html response
html = BeautifulSoup(raw_html, 'html.parser')

c = 0
most_act = []
gainers = []
losers = []

#Parsing the html response to get a list of all the tickers and stored in currosponding lists of most active, gainers and losers 
table = html.find('div', attrs = {'id':'wsod_hotStocks'}) 
for p in table.select('tr'):
    x = p.find('a', attrs = {'class':'wsod_symbol'})
    if(x != None):
        if c == 1:
            most_act.append(x.text + ' ' +  p.find('span').text)
        elif c == 2:
            gainers.append(x.text + ' ' +  p.find('span').text)
        elif c == 3:
            losers.append(x.text + ' ' +  p.find('span').text)
    else:
        c = c + 1

In [5]:
#This block creates a file named "final_beta.csv" that stores the ticker and its information
#This block reads all the tickers from the above list, gets the ticker information from "finance.yahoo.com"
#and stores it in the final_beta.csv file
#This block usually takes a while to run since it scraps data from the website multiple times for each ticker in the list
with open('final_beta.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    for y in most_act,gainers,losers:
        
        if y == gainers:
            first = 'Gainers'
        elif y == most_act:
            first = 'Most Active'
        elif y == losers:
            first = 'Losers'
        for x in y:
            ticker = x.split(maxsplit=1)[0]

            link = 'https://finance.yahoo.com/quote/' + ticker
            raw_html = simple_get(link)
            html = BeautifulSoup(raw_html, 'html5lib')
            l = []
            for p in html.select('td'):
                l.append(p.text)

            for i in range(len(l)):
                if l[i] == 'Previous Close':
                    close = l[i+1]
                elif l[i] == 'Open':
                    openv = l[i+1]
                elif l[i] == 'Volume':
                    vol = l[i+1]
                elif l[i] == 'Market Cap':
                    marcap = l[i+1]

            employee_writer.writerow([first,ticker,x,openv,close,vol,marcap])
    

In [6]:
#This block interacts with the users
#It shows all the tickers available in the the .csv file and asks the user for input
#It also asks the user if he wants to continue getting ticker information
#If the user inputs wrong ticker information,
#It prompts the user that the ticker does not exist and prompts the user to enter correct input
conti = True

while conti:
    print('Which stock are you interested in:')
    for y in most_act,gainers,losers:
        if y == most_act:
            print('\nMost Active')
        elif y == gainers:
            print('\nGainers')
        elif y == losers:
            print('\nLosers')
        for x in y:
            print(x)

    tickername = input('\n\nUser inputs: ').upper()
    gotit = False
    with open('final_beta.csv', newline = '') as games:
        game_reader = csv.reader(games, delimiter=',')
        for game in game_reader:
            if len(game) == 0:
                continue;
            if game[1] != tickername:
                continue;
            elif game[1] == tickername:
                gotit = True
                print('\nThe data for',game[2],'is the following:\n')
                print(game[2])
                print('OPEN: ',game[3])
                print('PREV CLOSE: ',game[4])
                print('VOLUME: ',game[5])
                print('MARKET CAP: ',game[6])
                break
                
    if not gotit:
        print('Ticker does not exist in the list above\nPlease enter a ticker from the list')
        conti = True
    else:
        if input('Do you want to continue?  Y/N: ').lower() == 'y':
            conti = True
        else:
            conti = False

Which stock are you interested in:

Most Active
AMD Advanced Micro Devices Inc
BAC Bank of America Corp
GE General Electric Co
T AT&T Inc
F Ford Motor Co
EXPE Expedia Group Inc
CRM Salesforce.Com Inc
INTC Intel Corp
MU Micron Technology Inc
AAPL Apple Inc

Gainers
EXPE Expedia Group Inc
LB L Brands Inc
PXD Pioneer Natural Resources Co
NBL Noble Energy Inc
CPRI Capri Holdings Ltd
EOG EOG Resources Inc
DVN Devon Energy Corp
CXO Concho Resources Inc
HAL Halliburton Co
MCHP Microchip Technology Inc

Losers
CRM Salesforce.Com Inc
JEC Jacobs Engineering Group Inc
NOW ServiceNow Inc
NRG NRG Energy Inc
CHTR Charter Communications Inc
HFC HollyFrontier Corp
ATVI Activision Blizzard Inc
STZ Constellation Brands Inc
PM Philip Morris International Inc
TTWO Take-Two Interactive Software Inc


User inputs: PXD

The data for PXD Pioneer Natural Resources Co is the following:

PXD Pioneer Natural Resources Co
OPEN:  125.15
PREV CLOSE:  123.49
VOLUME:  853,031
MARKET CAP:  21.579B
Do you want to contin